In [16]:
import numpy as np
import pandas as pd
import time 

np.random.seed(2) #reproducible 计算机产生一组伪随机数列，虽然是随机的，但每次运行的随机过程都是一样的

#创建几个lobal variables
N_STATES=6 #the length of the 1 dimensional world
ACTIONS=['left','right'] #两个可选动作
EPSILON=0.9 #ϵ - greedy中ϵ=0.9
ALPHA=0.1 #学习率
LAMBDA=0.9 #折扣系数
MAX_EPISODES=13 #只玩13回合
FRESH_TIME=0.03 #用time模块规定走一步花0.3秒

#建立Q表
def build_q_table(n_states,actions):#用pandas创建一个表格
    table=pd.DataFrame(
        np.zeros((n_states,len(actions))), #全0初始化
        columns=actions
    )
    return table
#build_q_table(N_STATES,ACTIONS) #行名是s,列名是a



#创建选动作的功能 This is how to choose an action
def choose_action(state,q_table): #根据目前所处的状态和Q表
    state_actions=q_table.iloc[state,:]
    if(np.random.uniform()>EPSILON) or ((state_actions==0).all()):
        action_name=np.random.choice(ACTIONS) #随机数大于0.9（10%的情况）或处于初始状态就随机选择ACTIONS里面的动作
    else:
        action_name=state_actions.idxmax() #随机数小于0.9（90%的情况）选择当前状态下动作估值大的动作
        # replace argmax to idxmax as argmax means a different function in newer version of pandas
    return action_name


#This is how agent will interact with the environment
def get_env_feedback(S,A): #现在的状态S，采取动作A，获得环境给的奖励值R，到达下一个状态S_
    if A=='right':
        if S==N_STATES-2:
            S_='terminal'
            R=1
        else:
            S_=S+1
            R=0
    else:
        R=0
        if S==0:
            S_=S
        else:
            S_=S-1
    return S_,R

#创建环境，怎么让探索者在这样一个一维环境中移动寻宝
def update_env(S, episode, step_counter):
    # This is how environment be updated
    env_list = ['-']*(N_STATES-1) + ['T']   # '---------T' our environment
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('\r{}'.format(interaction), end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)
        
#主循环 Main part of RL loop
def rl():
    q_table=build_q_table(N_STATES,ACTIONS)
    for episode in range(MAX_EPISODES): #从第一个回合到最后一个回合
        step_counter=0
        S=0 #初始情况把探索者放在最左边
        is_terminated=False #初始不是终止符(回合结束)
        update_env(S,episode,step_counter) #第一步更新一下环境
        while not is_terminated: #回合没有结束
            
            A=choose_action(S,q_table) #根据初始的state选择一个action
            S_,R=get_env_feedback(S,A) #根据state和action在环境中得到下一个state和reward
            q_predict=q_table.loc[S,A] #估计值
            if S_!='terminal':#探索者还没有寻到宝物（未到达终点）
                q_target=R+LAMBDA*q_table.iloc[S_,:].max() #真实值
            else:#探索者寻到宝物到达终点，没有下一步的状态，此时真实值就是reward
                q_target=R
                is_terminated=True #该episode结束
            
            q_table.loc[S,A]+=ALPHA*(q_target-q_predict)
            S=S_ #移向下一个状态
            
            update_env(S,episode,step_counter+1) #探索者走了每一步，再更新一下环境
            step_counter+=1
    return q_table


    
#运行
if __name__=='__main__':
    q_table=rl() #最终的q_table(学习好的Q表)
    print('\r\nQ-table:\n')#打印
    print(q_table)




                                
Q-table:

       left     right
0  0.000000  0.004320
1  0.000000  0.025005
2  0.000030  0.111241
3  0.000000  0.368750
4  0.027621  0.745813
5  0.000000  0.000000
